In [1]:
import torch
import os
import numpy as np
from dotenv import load_dotenv
from pinecone import Pinecone, PodSpec
from langchain.vectorstores import Pinecone as PineconeStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer
from langchain import HuggingFaceHub
import time

In [2]:
cache_dir="cache_dir"
load_dotenv('./.env')

True

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

if num_gpus > 0:
    print(f"GPUs available: {num_gpus}")
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
    print("There is no GPU available. Using CPU")

GPUs available: 1
GPU 0: NVIDIA RTX A4000


#### Leitura do arquivo e criação do splitter'

In [4]:
with open('docs/nat.txt') as f:
    clt = f.read()

# Criação de um objeto RecursiveCharacterTextSplitter para dividir o texto em pedaços
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10,
    length_function=len
)

In [5]:
chunks = text_splitter.create_documents([clt])
text_chunks = [doc.page_content for doc in chunks]

In [6]:
# Inicialize o modelo de embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Isso retornará uma lista de vetores
embeddings = model.encode(text_chunks, convert_to_tensor=False)  

# Adiciona zeros para estender cada vetor até a dimensão 1536
extended_embeddings = np.array([np.pad(emb, (0, 1536 - len(emb)), 'constant') for emb in embeddings])
print(extended_embeddings[0].shape)

(1536,)


In [7]:
pc = Pinecone(api_key=os.environ.get('PINECONE_API_KEY'))
indexes = pc.list_indexes()
index_name = 'natgpt'

In [8]:
for i in indexes:
    pc.delete_index(i['name'])
    print('Index encontrada e apagada: ' + i['name'])

if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=1536, metric='cosine', spec=PodSpec(environment=os.environ.get('PINECONE_ENV')))
    print('Index '+index_name+' criado')

Index encontrada e apagada: natgpt
Index natgpt criado


In [9]:
# Conecta na index criada
pc_index = pc.Index(index_name)

In [10]:
documents_to_insert = []
for i, embedding in enumerate(extended_embeddings):
    doc_id = f"{i}"
    documents_to_insert.append({"id": doc_id, "values": embedding.tolist()})

In [11]:
# Insere os documentos na index do Pinecone
pc_index.upsert(vectors=documents_to_insert)

# Aguarda 20 segundos para dar tempo de atualizar a Index no Pinecone
time.sleep(20)

### Busca por similaridade

In [12]:
# SentenceTransformer para gerar o embedding
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Inicialização do modelo para perguntas e respostas
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={
        "temperature": 0.5, 
        "max_new_tokens": 512,
        "debug": False,
        "top_k": 30,
        "top_p": 0.9,
        "repetition_penalty": 1.0,
    },
)

/linux-data/miniconda3/envs/linuxtips-llm/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [13]:
def perguntar(prompt, top_k=3, debug=False):
    # Gerando o vetor de consulta
    query_vector = model.encode(prompt, convert_to_tensor=False)
    
    # Adiciona zeros para estender cada vetor até a dimensão 1536
    padding_length = 1536 - len(query_vector)
    padded_vector = np.pad(query_vector, (0, padding_length), 'constant')
    print(padded_vector.shape)
    
    query_vector_list = padded_vector.tolist()
    query_result = pc_index.query(vector=query_vector_list, top_k=top_k)
    
    contexto = []
    for index, match in enumerate(query_result.matches):
        # print("ID: " + match.id)
        # print("Score: " + str(query_result.matches[index]['score']))
        # print("Text Value: " + text_chunks[int(match.id)])
        # print('-' * 50)
        contexto.append(text_chunks[int(match.id)])

    context_texts = contexto

    combined_context = " ".join(contexto)

    input_text = f"[INST]Responda em Português: {prompt}.\n\nContexto: {contexto}[/INST]"

    response = llm(input_text)

    if(debug == True):
        response = response + "\n\n\nDebug\n---------------------------------\n" + combined_context

    return response

In [18]:
print(
    perguntar(
        prompt='Quem são os coordenadores do nat?', top_k=3, debug=False
    )
)

(1536,)
 Os coordenadores do NAT são Marcela Cristina Ozório e Bernardo Fiterman Albano.


In [19]:
print(
    perguntar(
        prompt='Qual o site do nat?', top_k=3, debug=False
    )
)

(1536,)
 O site do NAT pode ser acessado por meio do endereço: https://nat.mpac.mp.br.


In [21]:
print(
    perguntar(
        prompt='Quais os principais setores do NAT', top_k=3, debug=False
    )
)

(1536,)
 Os principais setores do NAT são: Coordenação de Desenvolvimento de Sistemas, Observatório Criminal, LAB, Observatório de Políticas Públicas, Técinco-Científica.
